In [2]:
import streamlit as st
from streamlit_option_menu import option_menu
import easyocr
import mysql.connector
from PIL import Image
import pandas as pd
import numpy as np
import re
import io
import sqlite3

In [3]:
def img_to_txt(path):
    input_img = Image.open(path)

    #converting image to array format
    image_arr= np.array(input_img)

    reader = easyocr.Reader(['en'])
    text = reader.readtext(image_arr, detail=0)

    return text, input_img

In [4]:
# C:/Users/HP/Documents/project/BizCardx/1.png/
# C:/Users/HP/Documents/project/BizCardx/2.png/
# C:/Users/HP/Documents/project/BizCardx/3.png/
# C:/Users/HP/Documents/project/BizCardx/4.png/
# C:/Users/HP/Documents/project/BizCardx/5.png/

In [5]:
text_img, input_img = img_to_txt("C:/Users/HP/Documents/project/BizCardx/5.png/")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [6]:
def extract_text(texts):
    extract_dict = {"NAME":[],
                    "DESIGNATION":[],
                    "COMPANY NAME": [],
                    "CONTACT":[],
                    "EMAIL":[],
                    "WEBSITE":[],
                    "ADDRESS":[],
                    "PINCODE":[]}
    

    extract_dict["NAME"].append(texts[0])
    extract_dict["DESIGNATION"].append(texts[1])
    
    for i in range(2, len(texts)):
        if texts[i].startswith("+") or (texts[i].replace("-","").isdigit() and "-" in texts[i]):
            extract_dict["CONTACT"].append(texts[i])

        elif "@" in texts[i] and ".com" in texts[i]:
            extract_dict["EMAIL"].append(texts[i])

        elif "WWW" in texts[i] or "www" in texts[i]:
            small = texts[i].lower()
            extract_dict["WEBSITE"].append(small)

        elif "Tamil Nadu" in texts[i] or "TamilNadu" in  texts[i] or texts[i].isdigit():
            extract_dict["PINCODE"].append(texts[i])

        elif re.match(r'^[A-Za-z]',  texts[i]):
            extract_dict["COMPANY NAME"].append( texts[i])

        else:
            remove_colon = re.sub(r'[,;]', '', texts[i])
            extract_dict["ADDRESS"].append(remove_colon)

    for key,value in extract_dict.items():
        if len(value)>0:
            concadenate = " ".join(value)
            extract_dict[key] = [concadenate]
        
        else:
            value = "NA"
            extract_dict[key] = [value]

    return extract_dict

In [7]:
extract_text(text_img)

{'NAME': ['SANTHOSH'],
 'DESIGNATION': ['Technical Manager'],
 'COMPANY NAME': ['Sun Electricals'],
 'CONTACT': ['+123-456-1234'],
 'EMAIL': ['hello@Sun.com'],
 'WEBSITE': ['www.suncom'],
 'ADDRESS': ['NA'],
 'PINCODE': ['123 ABC St , Tirupur, TamilNadu; 641603']}

In [8]:
df = pd.DataFrame(extract_text(text_img))

In [9]:
df

,NAME,DESIGNATION,COMPANY NAME,CONTACT,EMAIL,WEBSITE,ADDRESS,PINCODE
0,SANTHOSH,Technical Manager,Sun Electricals,+123-456-1234,hello@Sun.com,www.suncom,NA,"123 ABC St , Tirupur, TamilNadu; 641603"


In [10]:
text_img

['SANTHOSH',
 'Technical Manager',
 '123 ABC St , Tirupur, TamilNadu;',
 '641603',
 '+123-456-1234',
 'hello@Sun.com',
 'www.Suncom',
 'Sun Electricals']

In [11]:
# converting image to bytes

Image_bytes= io.BytesIO()
input_img.save(Image_bytes, format= "PNG")    

Image_data = Image_bytes.getvalue()

# creating dictionary

data = {"IMAGE":[Image_data]}

df1 = pd.DataFrame(data)

concadenate_df = pd.concat([df,df1],axis=1)

In [12]:
import MySQLdb
#table creation

mydb = MySQLdb.connect(
  host="localhost",
  user="root",
  password="root",
  database="bizcardx"
)

cursor = mydb.cursor()

create_query = '''CREATE TABLE IF NOT EXISTS bizcard_details(name VARCHAR(255),
                                                           designation VARCHAR(255),
                                                           company_name VARCHAR(255),
                                                           contact VARCHAR(255),
                                                           email VARCHAR(255),
                                                           website TEXT,
                                                           address TEXT,
                                                           pincode VARCHAR(255),
                                                           image LONGBLOB)'''

cursor.execute(create_query)
mydb.commit()
